In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Part_Number'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=' + _ for _ in list_url]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = json.loads(requests.get(url,
                                               headers=UA.get_User_Agent_Requests(),
                                               proxies=Proxy.get_Proxy_Requests()).text)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for _ in resp['pp']['buyersGuides']:
                    list_vehicle.append(_['buyersGuideDesc'])

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': resp['pd']['basePart'],
                                         'Part_Type': resp['pd']['partDesc_en'],
                                         'Part_Type_Long': resp['pd']['partLongDesc_en'],
                                         'Brand': resp['pd']['brand'],
                                         'Pop': resp['pd']['pop'],
                                         'Per_Car': resp['pd']['qtyEach'],
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': 'https://ecatalog.smpcorp.com/V2/STD/#/partdetail/part/' + resp['pd']['basePart'],
                                         'Src': ''}])
                
                for name_value in resp['pp']['partSpecs']:
                    df_temp[name_value['attributeName_en'].lower()] = name_value['siteValue_en']
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：430

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS102M  <->  [ok] - 剩余数量：425
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS105M  <->  [ok] - 剩余数量：424
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS104M  <->  [ok] - 剩余数量：423
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS103M  <->  [ok] - 剩余数量：422
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS104R  <->  [ok] - 剩余数量：421
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS101M  <->  [ok] - 剩余数量：420
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS106M  <->  [ok] - 剩余数量：419
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS105R  <->  [ok] - 剩余数量：418
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=QS103R  <->  [ok] - 剩余数量：417
https://ecatalog.smpcorp.com/V2/STD/api/part/p

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM62  <->  [ok] - 剩余数量：347
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM61  <->  [ok] - 剩余数量：346
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM58A  <->  [ok] - 剩余数量：345
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM66A  <->  [ok] - 剩余数量：344
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM63  <->  [ok] - 剩余数量：343
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM65A  <->  [ok] - 剩余数量：342
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM66  <->  [ok] - 剩余数量：341
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM68A  <->  [ok] - 剩余数量：340
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM67  <->  [ok] - 剩余数量：339
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM149A  <->  [ok] - 剩余数量：269
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM144A  <->  [ok] - 剩余数量：268
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM155  <->  [ok] - 剩余数量：267
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM153  <->  [ok] - 剩余数量：266
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM152A  <->  [ok] - 剩余数量：265
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM45RA  <->  [ok] - 剩余数量：264
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM155A  <->  [ok] - 剩余数量：263
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM153A  <->  [ok] - 剩余数量：262
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM162  <->  [ok] - 剩余数量：261
https://ecatalog.smpcorp.com/V2/STD/api/part/part

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM294  <->  [ok] - 剩余数量：190
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM272  <->  [ok] - 剩余数量：189
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM295  <->  [ok] - 剩余数量：188
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM292  <->  [ok] - 剩余数量：187
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM300  <->  [ok] - 剩余数量：186
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM298  <->  [ok] - 剩余数量：185
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM266  <->  [ok] - 剩余数量：184
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM290  <->  [ok] - 剩余数量：183
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM301  <->  [ok] - 剩余数量：182
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM930K4  <->  [ok] - 剩余数量：112
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM1025  <->  [ok] - 剩余数量：111
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM1030K  <->  [ok] - 剩余数量：110
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM1040K  <->  [ok] - 剩余数量：109
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM950K4  <->  [ok] - 剩余数量：108
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM392  <->  [ok] - 剩余数量：107
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM1050K  <->  [ok] - 剩余数量：106
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM1060K  <->  [ok] - 剩余数量：105
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM336  <->  [ok] - 剩余数量：104
https://ecatalog.smpcorp.com/V2/STD/api/pa

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM2106VK4  <->  [ok] - 剩余数量：35
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM2105VK  <->  [ok] - 剩余数量：34
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM2115VK4  <->  [ok] - 剩余数量：33
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM3001K4  <->  [ok] - 剩余数量：32
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM2105VK4  <->  [ok] - 剩余数量：31
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM2115VK  <->  [ok] - 剩余数量：30
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM2116VK4  <->  [ok] - 剩余数量：29
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM3003K  <->  [ok] - 剩余数量：28
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=TPM3002K4  <->  [ok] - 剩余数量：27
https://ecatalog.smpcorp.com/V2/ST